Завдання: реалізувати алгоритм Q-learning для вирішення CartPole середовища OpenAi-gym
1.	Ознайомитись з OpenAI gym документацією https://gym.openai.com/docs/
2.	Ознайомитись з віртуальним середовищем CartPole-v1
3.	Реалізувати алгоритм Q-learning та навчити агента балансувати карт
4.	Зобразити у вигляді відео результат - балансуючу cart pole понад 100 фреймів
5.	Результат тренування зберегти у файл для можливості відтворення гри під час здачі лабораторної роботи. Ці ж артефакти завантажити на внс.

In [1]:
import numpy as np
import gym
import time
import math

In [3]:
env = gym.make("CartPole-v1")
"""Виставляємо дискретизацію простору значень середовища - найбільше варіантів для кутової швидкості"""
Observation = [30, 30, 30, 50]
"""Використовується для отримання дискретизованого стану(в нашому випадку 1 або 0)"""
np_array_win_size = np.array([0.25, 0.25, 0.01, 0.1])

"""Можемо вибирати чи тренувати агента чи брати збережену q-table(агент) з файлика"""
q = np.random.uniform(low=0, high=1, size=(Observation + [env.action_space.n]))
#q = np.loadtxt("q2.txt", delimiter=' ').reshape(30, 30, 30, 50, 2)

[2021-06-05 07:43:57,898] Making new env: CartPole-v1


In [4]:
"""Функція для отримання дискретного стану"""


def get_discrete_state(state):
    discrete_state = state / np_array_win_size + np.array([15, 10, 1, 10])
    return tuple(discrete_state.astype(int))


In [5]:
"""для тренування використовується ϵ-greedy exploration strategy, тобто спочатку ми більше досліджуємо світ, а потім, 
під кінець ми зменшує epsilon, щоб використовувати оптимальний варіант"""


def train(q_table):
    LEARNING_RATE = 0.1

    # визначає яку важливість ми надаємо для майбутніх винагород
    gamma = 0.95
    # кількість епізодів навчання
    EPISODES = 100000

    total = 0
    total_reward = 0
    prior_reward = 100

    # визначає чи ми досліджуємо світ(якщо близька до 1) чи використовуємо наявні дані з q-table
    epsilon = 1
    epsilon_decay_value = 0.99995
    """ цикл навчання"""
    for episode in range(EPISODES + 1):
        t0 = time.time()
        # отримуємо початковий стан середовища
        discrete_state = get_discrete_state(env.reset())
        done = False
        episode_reward = 0

        if episode % 10000 == 0:
            print("Episode: " + str(episode))
        # якщо стан середовища не закінчений(він закічнується, якщо пол має кут більше 12 градусів, чи карт змістився на
        # 2.4 від центру) то ми вибираємо дію зважаючи на значення епсілон
        while not done:

            if np.random.random() > epsilon:
                action = np.argmax(q_table[discrete_state])
            else:
                action = np.random.randint(0, env.action_space.n)
            # отримуємо новий стан середовища додаємо винагороду і якщо не закінчений епізод, то апдейтимо q-table
            new_state, reward, done, _ = env.step(action)
            episode_reward += reward
            new_discrete_state = get_discrete_state(new_state)

            if episode % 2000 == 0:
                env.render()

            if not done:
                max_future_q = np.max(q_table[new_discrete_state])
                current_q = q_table[discrete_state + (action,)]
                new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + gamma * max_future_q)
                q_table[discrete_state + (action,)] = new_q

            discrete_state = new_discrete_state
        # Тут, якщо епсілон відносно велике і ми досягаємо винагороди в prior_reward(100)(тобто виживаємо хоча б 100
        # фреймів), то можна і поменше досліджувати середовище - тому зменшуємо епсілон
        if epsilon > 0.05:
            if episode_reward > prior_reward and episode > 10000:
                epsilon = math.pow(epsilon_decay_value, episode - 10000)

                if episode % 500 == 0:
                    print("Epsilon: " + str(epsilon))
        # просто для виводу
        t1 = time.time()
        episode_total = t1 - t0
        total = total + episode_total

        total_reward += episode_reward
        prior_reward = episode_reward

        if episode % 1000 == 0:
            mean = total / 1000
            print("Time Average: " + str(mean))
            total = 0

            mean_reward = total_reward / 1000
            print("Mean Reward: " + str(mean_reward))
            total_reward = 0


In [ ]:
"""У випадку тренування розкоментовуємо, щоб зберегти таблицю"""
#train(q_table=q)
#np.savetxt("q.txt", q.reshape(-1, 2), delimiter=' ')
discrete_state_ = get_discrete_state(env.reset())
for i in range(1000):
    env.render()
    action_ = np.argmax(q[discrete_state_])
    state_, _, d, _ = env.step(action_)  # take a random action
    if d:
        print(i)
        break
    else:
        discrete_state_ = get_discrete_state(state_)
env.close()


Episode: 0
Time Average: 0.001116889238357544
Mean Reward: 0.052
Time Average: 0.000694732666015625
Mean Reward: 22.823
Time Average: 0.0008283312320709229
Mean Reward: 22.464
Time Average: 0.0006723275184631347
Mean Reward: 22.407
Time Average: 0.0013739967346191406
Mean Reward: 21.617
Time Average: 0.0006691486835479736
Mean Reward: 22.595
Time Average: 0.0009544410705566406
Mean Reward: 22.161
Time Average: 0.0006527175903320312
Mean Reward: 21.781
Time Average: 0.0008146114349365235
Mean Reward: 22.397
Time Average: 0.0006937730312347412
Mean Reward: 22.415
Episode: 10000
Time Average: 0.0009156770706176758
Mean Reward: 22.932
Epsilon: 0.9753093024395111
Epsilon: 0.9512282354250458
Time Average: 0.0006836295127868652
Mean Reward: 23.129
Epsilon: 0.9048351558698463
Time Average: 0.0011341142654418945
Mean Reward: 23.818
Time Average: 0.0007776367664337158
Mean Reward: 25.546
Epsilon: 0.818726659298009
Time Average: 0.0011361756324768067
Mean Reward: 26.931
Epsilon: 0.798511726968572